In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

from utils import load_images_from_folder, show_images, BGR_to_RBG 
from board_detection import get_points, warp_image,get_contour

In [ ]:
folder = './testy'
images = load_images_from_folder(folder)
show_images(BGR_to_RBG(images),10,200,(len(images),1))

In [ ]:
output_images = []
for img in images:
    contour_img,cnt = get_contour(img)
    pts, size = get_points(cnt)
    warped_img = warp_image(img, pts,size)
    output_images.append(warped_img)
show_images(BGR_to_RBG(images),10,200,(len(images),1))

In [ ]:
import image_slicer
for i in range(len(output_images)):
    file = './warped_images/warped'+str(i)+'.jpg'
    cv2.imwrite(file,output_images[i])
    tiles = image_slicer.slice(file,225,save=False)
    image_slicer.save_tiles(tiles, directory='./sliced_letters',prefix='slice'+str(i), format='png')